In [10]:
from folium import plugins , GeoJson, FeatureGroup, IFrame
from folium import Icon, Element, CircleMarker, Map, Popup, LayerControl , Marker

import pandas as pd

In [2]:
df = pd.read_csv('../Data/modified_Thawra_2019-2020 - Main.csv', header=1, parse_dates=['Startdate'])

In [3]:
df = df[['Startdate','gpslatend', 'gpslongend','demandtypeone']].sample(100)
df

,Startdate,gpslatend,gpslongend,demandtypeone
875,2019-11-05,33.635717,35.568545,Thawra 2019
1440,2019-11-17,33.398551,35.685407,Thawra 2019
2091,2019-12-11,33.838197,35.912357,Thawra 2019
599,2019-10-26,34.543452,36.077207,Thawra 2019
1786,2019-12-01,34.061857,35.684162,Corruption
...,...,...,...,...
1251,2019-11-13,33.467391,35.315903,Thawra 2019
1306,2019-11-14,34.391850,35.847988,Thawra 2019
2223,2019-12-18,33.554258,35.377501,Labor
624,2019-10-27,33.272353,35.203508,Thawra 2019


In [4]:
df = df.reset_index().drop(columns='index').dropna()
df

,Startdate,gpslatend,gpslongend,demandtypeone
0,2019-11-05,33.635717,35.568545,Thawra 2019
1,2019-11-17,33.398551,35.685407,Thawra 2019
2,2019-12-11,33.838197,35.912357,Thawra 2019
3,2019-10-26,34.543452,36.077207,Thawra 2019
4,2019-12-01,34.061857,35.684162,Corruption
...,...,...,...,...
95,2019-11-13,33.467391,35.315903,Thawra 2019
96,2019-11-14,34.391850,35.847988,Thawra 2019
97,2019-12-18,33.554258,35.377501,Labor
98,2019-10-27,33.272353,35.203508,Thawra 2019


In [5]:
def color_demand(demand):
    d = {'Corruption':'red', 'Politics (local)':'blue', 'Thawra 2019':'green',
       'Economic/Financial':'purple', 'Human rights / civic freedoms':'orange', 'Social':'darkred',
       'Politics (national)':'lightred', "Women's rights":'pink',
       'Politics (regional/international)':'darkblue', 'Religion/Sectarian':'red',
       'Security':'darkgreen', 'Labor':'gray', 'Environmental':'darkred'}
    color= d[demand]
    return color
df['color'] = df['demandtypeone'].apply(color_demand)

In [6]:
color_demand('Environmental')

'darkred'

In [7]:
df

,Startdate,gpslatend,gpslongend,demandtypeone,color
0,2019-11-05,33.635717,35.568545,Thawra 2019,green
1,2019-11-17,33.398551,35.685407,Thawra 2019,green
2,2019-12-11,33.838197,35.912357,Thawra 2019,green
3,2019-10-26,34.543452,36.077207,Thawra 2019,green
4,2019-12-01,34.061857,35.684162,Corruption,red
...,...,...,...,...,...
95,2019-11-13,33.467391,35.315903,Thawra 2019,green
96,2019-11-14,34.391850,35.847988,Thawra 2019,green
97,2019-12-18,33.554258,35.377501,Labor,gray
98,2019-10-27,33.272353,35.203508,Thawra 2019,green


In [22]:
# Create a map object
m = Map(location=[33.905074, 35.744929])

# Create the data for the TimestampedGeoJson plugin
data = {
    'type': 'FeatureCollection',
    'features': []
}

for i, row in df.iterrows():
    t, lat, lon, demand, color = row['Startdate'], row['gpslatend'], row['gpslongend'], row['demandtypeone'], row['color']
    # CircleMarker(location=[lat, lon],
    #              radius=5,
    #              color=color,
    #              popup=demand,
    #              fill_color=color,
    #              fill=True).add_to(m)
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [lon, lat],
        },
        'properties': {
            'time': t.strftime('%Y-%m-%dT%H:%M:%S'),
            'value':demand
        }
    }
    data['features'].append(feature)

# Add a timeline to the map
timeline = plugins.TimestampedGeoJson(data, period='P1D', add_last_point=False).add_to(m)

# # Add a timeline to the map

# coordinates = []
# time = []
# for indx in df.index:
#     coordinates.append([df['gpslatend'][indx], df['gpslongend'][indx]])
#     time.append(df['Startdate'][indx].strftime('%Y-%m-%dT%H:%M:%S'))

# timeline = plugins.TimestampedGeoJson({
#     'type': 'FeatureCollection',
#     'features': [
#         {
#             'type': 'Feature',
#             'geometry': {
#                 'type': 'Point',
#                 'coordinates': coordinates,
#             },
#             'properties': {
#                 'time': time
#             }
#         }
#     ]
# }, period='PT1M', add_last_point=True).add_to(m)

# Display the map
m

In [19]:
# layers = {}
# for value in df['value'].unique():
#     # Create the data for the TimestampedGeoJson plugin
#     data = {
#         'type': 'FeatureCollection',
#         'features': []
#     }
#     for i, row in df[df['value'] == value].iterrows():
#         lon, lat, t, value = row['lon'], row['lat'], row['time'], row['value']
#         feature = {
#             'type': 'Feature',
#             'geometry': {
#                 'type': 'Point',
#                 'coordinates': [lon, lat],
#             },
#             'properties': {
#                 'time': t.strftime('%Y-%m-%dT%H:%M:%S'),
#                 'value': value
#             }
#         }
#         data['features'].append(feature)


In [75]:
# Create a map object

m2 = Map(location=[33.905074, 35.744929])

# Create a list of GeoJson objects, one for each value

geojson_list = []

for demand in df['demandtypeone'].unique():
    # Create the data for the GeoJson object
    data = {
        'type': 'FeatureCollection',
        'features': []
    }
    for i, row in df[df['demandtypeone'] == demand].iterrows():
        lon, lat, t, color = row['gpslongend'], row['gpslatend'], row['Startdate'], row['color']
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',  # Set the type to something other than 'point'
                'coordinates': [lon, lat],  # Set the coordinates of the feature
            },
            'properties': {
                'time': t.strftime('%Y-%m-%dT%H:%M:%S'),
                'color': color,
                'value':demand
            }
        }
        # Add a popup to the feature
        popup = Popup(demand)
        feature['properties']['popup'] = str(popup)
        data['features'].append(feature)



    # Create a GeoJson object and add it to the map
    def style_function(feature):
        color = feature['properties']['color']
        return {'color': color}

    geojson = GeoJson(data, style_function=style_function).add_to(m2)
   
    geojson_list.append(geojson)

# LayerControl(position='topright', collapsed=False).add_to(m2)

timeline = plugins.TimestampedGeoJson(data, period='P1D', add_last_point=True).add_to(m2)

# Display the map
m2

In [40]:
df

,Startdate,gpslatend,gpslongend,demandtypeone,color
0,2019-10-21,33.751043,35.450802,Thawra 2019,green
1,2020-01-24,33.897500,35.502263,Corruption,red
2,2020-01-28,33.556278,35.376892,Politics (regional/international),darkblue
3,2019-11-13,33.687647,35.520891,Security,darkgreen
4,2020-01-14,33.561481,35.342811,Economic/Financial,purple
...,...,...,...,...,...
95,2019-10-18,33.895782,35.567336,Thawra 2019,green
96,2019-11-28,34.544208,36.079792,Thawra 2019,green
97,2020-01-27,33.876134,35.507333,Thawra 2019,green
98,2019-12-02,33.874768,35.507415,Economic/Financial,purple


In [56]:
df['demandtypeone'] = df['demandtypeone'].astype(str)
demands = df['demandtypeone'].unique()

In [74]:
# Create a map object
m3 = Map(location=[33.905074, 35.744929])

# Create a feature group for each demand
fg_types = {demand: FeatureGroup(name=demand) for demand in df['demandtypeone'].unique()}

# Add a marker for each point
for _, row in df.iterrows():
    if row['demandtypeone'] in fg_types:
        CircleMarker(
            location=[row['gpslatend'], row['gpslongend']],
            radius=8,
            color=row['color'],
            fill=True,
            fill_opacity=0.7
        ).add_to(fg_types[row['demandtypeone']])
    
# Add the feature groups to the map
control = ''

for demand, fg in fg_types.items():
    fg.add_to(m3)
    control += f'<input type="checkbox" checked="True" onclick="{fg._name}.remove_from(mymap)" > {fg._name} <br>'
IFrame(control, width=100, height=200).add_to(m3)




# Create a time slider
plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': [
        {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': [row['gpslongend'],row['gpslatend']]
            },
            'properties': {
                'time': row['Startdate'].strftime('%Y-%m-%dT%H:%M:%S'),
                'type': row['demandtypeone'],
                'icon': 'circle'
            }
        } for _, row in df.iterrows()
    ]
}, period='P1D', add_last_point=True, auto_play=False, loop=True, max_speed=1, loop_button=True, date_options='YYYY-MM-DD', time_slider_drag_update=True).add_to(m3)

# Create a control to show/hide each demand
LayerControl(collapsed=False).add_to(m3)

# Show the map
m3


In [61]:
print(fg_types.keys())


dict_keys(['Thawra 2019', 'Corruption', 'Politics (regional/international)', 'Security', 'Economic/Financial', 'Human rights / civic freedoms', "Women's rights", 'Environmental', 'Social', 'Religion/Sectarian'])


In [8]:
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster
from ipywidgets import interact


In [11]:


# Create a function to filter and plot events by month
def plot_events_by_month(month):
    filtered_df = df[df['Startdate'].dt.month == month]
    
    # Create a map centered at a default location, e.g., (0, 0)
    m = Map(location=[0, 0], zoom_start=2)

    # Create a marker cluster for better performance
    marker_cluster = MarkerCluster().add_to(m)

    # Plot each event as a marker on the map
    for index, row in filtered_df.iterrows():
        Marker([row['gpslatend'], row['gpslongend']], popup=row['demandtypeone']).add_to(marker_cluster)

    # Display the map
    return m

# Create a dropdown widget to select the month
months = df['Startdate'].dt.month.unique()
interact(plot_events_by_month, month=months)

# Show the map
plt.show()


interactive(children=(Dropdown(description='month', options=(11, 12, 10, 1, 2, 3, 9), value=11), Output()), _d…